In [2]:
from elasticsearch8 import Elasticsearch
import json
def config(k):
    with open(f'/configs/default/shared-data/{k}', 'r') as f:
        return f.read()

def main():
    try:
        es = Elasticsearch (
                'https://127.0.0.1:9200',
                request_timeout=60,
                verify_certs= False,
                ssl_show_warn= False,
                basic_auth=('elastic','elastic')
            )

        query = {
        "size":100,
            "_source": ["_id","phn_name","phn_code","dths_rspt3"],
            "query": {
                "match_all": {}
            }
        }


        result = es.search(index="premature_mortality", body=query)
        docs = result['hits']['hits']
        features = {}
        for doc in docs:
            _id = doc['_id']
            death_ratio = doc['_source']['dths_rspt3']/100000
            phn_name = doc['_source']['phn_name']
            phn_code = doc['_source']['phn_code']
            features[_id] = {
                '_id': _id,
                'death_ratio':death_ratio,
                'phn_code': phn_code,
                'PHNName': phn_name
            }

        ids = list(features.keys())
        
        aggregation_data = []
        for id in ids:
            print("searching: ", id)
            query = {
                "size": 10000,
                    "query": {
                        "bool": {
                            "must": [
                                {
                                    "exists": {
                                        "field": "geometry"
                                    }
                                }
                            ],
                            "filter": {
                                "geo_shape": {
                                    "geometry": {
                                        "indexed_shape": {
                                            "index": "premature_mortality",
                                            "id": id,
                                            "path": "geometry"
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            # query = {
            #     "_source": False,
            #     "query": {
            #     "bool": {
            #         "must": [
            #             {
            #             "exists": {
            #                 "field": "geometry"
            #             }
            #             }
            #         ],
            #         "filter": {
            #             "geo_shape": {
            #                 "geometry": {
            #                     "indexed_shape": {
            #                     "index": "premature_mortality",
            #                     "id": id,
            #                     "path": "geometry"
            #                     }
            #                 }
            #             }
            #         }
            #     }
            # },
            # "aggs": {
            #     "dataName_avg": {
            #         "terms": {
            #             "field": "dataName"
            #         },
            #         "aggs": {
            #             "avg_value": {
            #                 "avg": {
            #                     "field": "averageValue"
            #                 }
            #             }
            #         }
            #     }
            # }
            # }

            result2 = es.search(index="air-qualities", body=query)
            aggs_data = result2['hits']['hits']
            for data in aggs_data:
                if data['_source']['averageValue']!=None:
                    aggregation_data.append({
                        '_id': id,
                        'dataName': data['_source']['dataName'],
                        'avgValue': data['_source']['averageValue']
                    })


        unique_data_names = set()
        for data in aggregation_data:
            unique_data_names.add(data['dataName'])

        for id in ids:
            for data_name in unique_data_names:
                values = []
                for data in aggregation_data:
                    if data['_id'] == id and data['dataName'] == data_name:
                        values.append(data['avgValue'])
                features[id][data_name] = values
                
        response = {
            'status_code': 200,
            'data': list(features.values())
        }
        return json.dumps(response)
    except Exception as e:
        error_response = {'status_code': 500, 'error_message': str(e)}
        return json.dumps(error_response)

print(main())

searching:  HDczdY8B6E3HLROlX8Pq
searching:  GTczdY8B6E3HLROlN8Mz
searching:  HTczdY8B6E3HLROlacO3
searching:  ITczdY8B6E3HLROlhsOQ
searching:  HzczdY8B6E3HLROldsPG
searching:  IzczdY8B6E3HLROlk8ML
searching:  FzczdY8B6E3HLROlMsMR
searching:  GDczdY8B6E3HLROlMsMR
searching:  HjczdY8B6E3HLROlccOC
searching:  IDczdY8B6E3HLROlfMPa
searching:  GjczdY8B6E3HLROlQMM8
searching:  GzczdY8B6E3HLROlTMO5
searching:  IjczdY8B6E3HLROljMOu
searching:  JDczdY8B6E3HLROltcMf
searching:  JTczdY8B6E3HLROlucMS
searching:  JjczdY8B6E3HLROlu8NQ
searching:  JzczdY8B6E3HLROlvsPC
searching:  KDczdY8B6E3HLROlxMN_
searching:  KTczdY8B6E3HLROlz8Pw
searching:  KjczdY8B6E3HLROl38Mp
searching:  Kzc0dY8B6E3HLROlGsOk
searching:  LDc0dY8B6E3HLROlIsPM
searching:  LTc0dY8B6E3HLROlIsPM
searching:  Ljc0dY8B6E3HLROlIsPM
searching:  Lzc0dY8B6E3HLROlKcMK
searching:  MDc0dY8B6E3HLROlLcOd
searching:  MTc0dY8B6E3HLROlPMNi
searching:  Mjc0dY8B6E3HLROlU8MH
searching:  Mzc0dY8B6E3HLROlYcNN
searching:  NDc0dY8B6E3HLROlcsOi
searching: